In [1]:
#!pip install transformers datasets

  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached cffi-1.17.1-cp310-cp310-win_amd64.whl.metadata (1.6 kB)
  Using cached Cython-3.0.11-cp310-cp310-win_amd64.whl.metadata (3.2 kB)
  Using cached hydra_core-1.0.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl.metadata (3.0 kB)
  Using cached omegaconf-2.0.5-py3-none-any.whl.metadata (3.0 kB)
  Using cached omegaconf-2.0.4-py3-none-any.whl.metadata (3.0 kB)
  Using cached omegaconf-2.0.3-py3-none-any.whl.metadata (3.0 kB)
  Using cached omegaconf-2.0.2-py3-none-any

Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/d0/eb/9d63ce09dd8aa85767c65668d5414958ea29648a0eec80a4a7d311ec2684/omegaconf-2.0.6-py3-none-any.whl (from fairseq) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    PyYAML (>=5.1.*)
            ~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/e5/f6/043b6d255dd6fbf2025110cea35b87f4c5100a181681d8eab496269f0d5b/omegaconf-2.0.5-py3-none-any.whl (from fairseq) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    PyYAML (>=5.1.*)
            ~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/92/b1/4f3023143436f12c98bab53f0b3db617bd18a7d223627d5030e13a7b4fc2/omegaconf-2.0.4-py3-none-any.whl (from fairseq) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    PyYAML (>=5.1.*)
        

In [2]:
import torch
import torch.nn as nn
from transformers import XLMRobertaConfig, XLMRobertaModel, AutoTokenizer, AutoModel

## Test without module

In [4]:
# Load tokenizer and model
model_checkpoint = "distilbert-base-uncased"
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModel.from_pretrained(model_name)
text = "This is an example sentence."
tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**tokens)

# Extract features from the last hidden layer
last_hidden_state = outputs.last_hidden_state

C:\Users\jhegu\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
tokens
outputs
last_hidden_state
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

## Class for XLM roberta Embedding

In [19]:
class XLM_Embedding(nn.Module):
    """
    Inputs: embedding dimension for the words, string to embed, max length of the titles

    Output: Embedding of the string word for word with the attention mask if needed"""
    def __init__(self, model = "xlm-alberta-base"):
        super(XLM_Embedding, self).__init__()
        
        self.tokenizer  = AutoTokenizer.from_pretrained(model)
        self.model = AutoModel.from_pretrained(model)

    def forward(self, text, max_len):
        assert max(len(sentence) for sentence in text)<=max_len
        tokens = self.tokenizer(text, return_tensors = "pt", padding= True, truncation = True)
        #apply the model
        res = self.model(**tokens)
        last_hidden_state = res.last_hidden_state
        #if you want the attention mask as output, you can uncomment the next line and add the attention mask in the return
        # attention_mask = tokens["attention_mask"]
        #return only the last state which contains the dense representations of each token
        return last_hidden_state
        
embedder = XLM_Embedding("xlm-roberta-base")
sentence_embedding = embedder("This is a sentence to try the model",30)
print(sentence_embedding)
print(sentence_embedding.shape)
        

tensor([[[ 0.0887,  0.0626,  0.0622,  ..., -0.0749,  0.0795, -0.0274],
         [-0.0762, -0.0833,  0.0425,  ..., -0.0128, -0.0034,  0.1306],
         [-0.0448, -0.0070,  0.0241,  ...,  0.2206,  0.0091,  0.0963],
         ...,
         [-0.0385, -0.0817,  0.0115,  ..., -0.0211,  0.0403,  0.2267],
         [-0.0579,  0.0076,  0.0340,  ..., -0.0997,  0.0186, -0.1023],
         [ 0.0810,  0.0547,  0.0109,  ..., -0.1423,  0.0191,  0.0099]]],
       grad_fn=<NativeLayerNormBackward0>)
torch.Size([1, 10, 768])
